# Deploying a Model Using SageMaker Endpoints

This notebook demonstrates how to register a model in the SageMaker Model Registry and then deploy it as a SageMaker Endpoint.

We'll walk through the following steps:
1. Set up necessary imports and configurations.
2. Register a model in the SageMaker Model Registry.
3. Retrieve the model from the SageMaker Model Registry.
4. Create an endpoint configuration.
5. Deploy the model as a SageMaker Endpoint.
6. Make predictions using the endpoint.

This example assumes that the model is a linear regression model trained using the Linear Learner algorithm and is for house price prediction based on input data in CSV format.

In [ ]:
# Step 1: Import necessary libraries
import boto3
from sagemaker import get_execution_role
from sagemaker.session import Session

# Define your SageMaker session and role
sagemaker_session = Session()
role = get_execution_role()

# Define S3 bucket and prefix
bucket = 'your-s3-bucket-name'
prefix = 'path/to/your/model'

## Step 2: Register a Model in the SageMaker Model Registry
We'll upload a model artifact to S3, specify inference image URI, and create a model package group to register the model.

In [ ]:
# Upload the model artifact to S3
model_artifact = f's3://{bucket}/{prefix}/model.tar.gz'

# Define inference container image URI
inference_image_uri = '683313688378.dkr.ecr.us-east-1.amazonaws.com/linear-learner:latest'

# Create a model package group
model_package_group_name = 'house-price-prediction-group'
sagemaker_client = boto3.client('sagemaker')

try:
    sagemaker_client.create_model_package_group(
        ModelPackageGroupName=model_package_group_name,
        ModelPackageGroupDescription='Model group for house price prediction'
    )
    print(f'Created Model Package Group: {model_package_group_name}')
except sagemaker_client.exceptions.ResourceInUse:
    print(f'Model Package Group {model_package_group_name} already exists.')

# Register the model
model_package_response = sagemaker_client.create_model_package(
    ModelPackageGroupName=model_package_group_name,
    ModelPackageDescription='Linear regression model for house price prediction',
    InferenceSpecification={
        'Containers': [
            {
                'Image': inference_image_uri,
                'ModelDataUrl': model_artifact
            }
        ],
        'SupportedContentTypes': ['text/csv'],
        'SupportedResponseMIMETypes': ['text/csv']
    },
    ModelApprovalStatus='Approved'
)

model_package_arn = model_package_response['ModelPackageArn']
print(f'Registered Model Package ARN: {model_package_arn}')

## Step 3: Retrieve the Model from the SageMaker Model Registry
We will retrieve the registered model package from the model package group.

In [ ]:
# Define your model package group name
# (This was already defined earlier as model_package_group_name)

# Get the latest approved model package
response = sagemaker_client.list_model_packages(
    ModelPackageGroupName=model_package_group_name,
    ModelApprovalStatus='Approved',
    SortBy='CreationTime',
    SortOrder='Descending'
)

latest_model_package_arn = response['ModelPackageSummaryList'][0]['ModelPackageArn']
print(f'Latest Approved Model Package ARN: {latest_model_package_arn}')

## Step 4: Create an Endpoint Configuration
The endpoint configuration specifies the model to use and the instance type for deployment.

In [ ]:
# Define endpoint configuration name
endpoint_config_name = 'house-price-endpoint-config'

# Create the endpoint configuration
sagemaker_client.create_endpoint_config(
    EndpointConfigName=endpoint_config_name,
    ProductionVariants=[
        {
            'VariantName': 'AllTraffic',
            'ModelName': latest_model_package_arn.split('/')[-1],
            'InstanceType': 'ml.m5.large',
            'InitialInstanceCount': 1
        }
    ]
)
print(f'Created endpoint configuration: {endpoint_config_name}')

## Step 5: Create the Endpoint
The endpoint is the actual service that serves predictions using your deployed model.

In [ ]:
# Define endpoint name
endpoint_name = 'house-price-prediction-endpoint'

# Create the endpoint
sagemaker_client.create_endpoint(
    EndpointName=endpoint_name,
    EndpointConfigName=endpoint_config_name
)
print(f'Creating endpoint: {endpoint_name}')

## Step 6: Test the Endpoint
Now that the endpoint is deployed, we can send input data for inference and get predictions.

In [ ]:
import json
import numpy as np

# Prepare sample input data
input_data = '2000,3,2,1'  # Example CSV input: square footage, bedrooms, bathrooms, garage

# Create runtime client
runtime_client = boto3.client('sagemaker-runtime')

# Make the prediction
response = runtime_client.invoke_endpoint(
    EndpointName=endpoint_name,
    ContentType='text/csv',
    Body=input_data
)

# Decode the response
prediction = response['Body'].read().decode('utf-8')
print(f'Predicted house price: {prediction}')